In [2]:
import pandas as pd
import numpy as np
import json
import re
from typing import List, Dict, Tuple, Set, Optional, Any
import networkx as nx
from collections import defaultdict, deque
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

class BacklogPrioritizer:
    def __init__(self, 
                 decay_factor: float = 0.6,
                 cv_folds: int = 5,
                 random_state: int = 42):
        """
        Algoritmo de priorização de backlog com ML e análise de grafos
        
        Args:
            decay_factor: Fator de decaimento para impacto transitivo de dependências
            cv_folds: Número de folds para cross-validation
            random_state: Seed para reprodutibilidade
        """
        self.decay_factor = decay_factor
        self.cv_folds = cv_folds
        self.random_state = random_state
        self.model = None
        self.label_encoders = {}
        self.feature_importance = None
        
    def read_and_standardize(self, file_path: str) -> pd.DataFrame:
        """Lê e padroniza o CSV de entrada"""
        # Detecção robusta de separador e encoding
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='latin1')
            except:
                df = pd.read_csv(file_path, encoding='cp1252')
        
        # Se não funcionou com vírgula, tenta outros separadores
        if len(df.columns) == 1:
            for sep in [';', '\t']:
                try:
                    df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                    if len(df.columns) > 1:
                        break
                except:
                    continue
        
        # Padronizar nomes das colunas
        column_mapping = {
            'issueid': 'id',
            'prioridade score': 'roi_ajustado',
            'prioridade_score': 'roi_ajustado',
            'storypoints': 'story_points',
            'story_points': 'story_points',
            'dependencias': 'deps',
            'dependencies': 'deps'
        }
        
        df.columns = df.columns.str.strip().str.lower()
        df = df.rename(columns=column_mapping)
        
        # Validar colunas obrigatórias
        required_cols = ['id', 'roi_ajustado', 'story_points', 'deps']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Colunas obrigatórias ausentes: {missing_cols}")
        
        # Converter tipos
        df['roi_ajustado'] = pd.to_numeric(df['roi_ajustado'], errors='coerce').fillna(0.0)
        df['story_points'] = pd.to_numeric(df['story_points'], errors='coerce')
        
        # Tratar nulos em story_points
        if df['story_points'].isna().all():
            df['story_points'] = 3.0
        else:
            df['story_points'].fillna(df['story_points'].median(), inplace=True)
        
        # Parsear dependências
        df['deps'] = df['deps'].apply(self._parse_dependencies)
        
        # Remover auto-dependências
        df['deps'] = df.apply(lambda row: [dep for dep in row['deps'] if dep != row['id']], axis=1)
        
        return df
    
    def _parse_dependencies(self, deps_str: Any) -> List[str]:
        """Parseia string de dependências em lista"""
        if pd.isna(deps_str) or deps_str == '':
            return []
        
        deps_str = str(deps_str).strip()
        if not deps_str:
            return []
        
        # Tentar parsear como JSON
        try:
            if deps_str.startswith('[') and deps_str.endswith(']'):
                return json.loads(deps_str)
        except:
            pass
        
        # Parsear com separadores
        deps = []
        for sep in [',', ';']:
            if sep in deps_str:
                deps = [dep.strip().strip('"\'') for dep in deps_str.split(sep)]
                break
        else:
            deps = [deps_str.strip().strip('"\'')]
        
        return [dep for dep in deps if dep]
    
    def build_dependency_graph(self, df: pd.DataFrame) -> Tuple[nx.DiGraph, Dict]:
        """Constrói grafo de dependências"""
        G = nx.DiGraph()
        external_deps = {}
        
        # Adicionar todos os nós
        valid_ids = set(df['id'].astype(str))
        G.add_nodes_from(valid_ids)
        
        # Adicionar arestas
        for _, row in df.iterrows():
            item_id = str(row['id'])
            external_deps[item_id] = []
            
            for dep in row['deps']:
                dep = str(dep)
                if dep in valid_ids:
                    # Aresta: dependência → item (pré-requisito destrava item)
                    G.add_edge(dep, item_id)
                else:
                    external_deps[item_id].append(dep)
        
        # Detectar e quebrar ciclos
        if not nx.is_directed_acyclic_graph(G):
            self._break_cycles(G, df)
        
        return G, external_deps
    
    def _break_cycles(self, G: nx.DiGraph, df: pd.DataFrame) -> None:
        """Quebra ciclos removendo arestas com menor prioridade"""
        roi_dict = dict(zip(df['id'].astype(str), df['roi_ajustado']))
        
        while not nx.is_directed_acyclic_graph(G):
            try:
                cycle = nx.find_cycle(G)
                # Encontrar aresta com menor prioridade no ciclo
                min_weight = float('inf')
                edge_to_remove = None
                
                for u, v in cycle:
                    weight = roi_dict.get(u, 0) + roi_dict.get(v, 0)
                    if weight < min_weight:
                        min_weight = weight
                        edge_to_remove = (u, v)
                
                if edge_to_remove:
                    G.remove_edge(*edge_to_remove)
                    print(f"Ciclo detectado. Removida aresta: {edge_to_remove}")
                else:
                    break
            except nx.NetworkXNoCycle:
                break
    
    def calculate_graph_features(self, df: pd.DataFrame, G: nx.DiGraph) -> pd.DataFrame:
        """Calcula features do grafo"""
        result_df = df.copy()
        roi_dict = dict(zip(df['id'].astype(str), df['roi_ajustado']))
        
        # Inicializar colunas
        result_df['in_degree'] = 0
        result_df['out_degree'] = 0
        result_df['dep_impact'] = 0.0
        result_df['degree_centrality'] = 0.0
        result_df['deps_ok'] = True
        
        # Calcular métricas
        centrality = nx.degree_centrality(G.to_undirected()) if len(G) > 1 else {}
        
        for idx, row in result_df.iterrows():
            item_id = str(row['id'])
            
            # Graus
            in_deg = G.in_degree(item_id)
            out_deg = G.out_degree(item_id)
            
            result_df.loc[idx, 'in_degree'] = in_deg
            result_df.loc[idx, 'out_degree'] = out_deg
            result_df.loc[idx, 'deps_ok'] = (in_deg == 0)
            result_df.loc[idx, 'degree_centrality'] = centrality.get(item_id, 0.0)
            
            # Impacto de dependências (soma do ROI dos dependentes diretos)
            dependents = list(G.successors(item_id))
            dep_impact = sum(roi_dict.get(dep, 0) for dep in dependents)
            result_df.loc[idx, 'dep_impact'] = dep_impact
        
        return result_df
    
    def train_ml_model(self, df: pd.DataFrame) -> Dict:
        """Treina modelo de ML para priorização"""
        # Selecionar features
        base_features = ['story_points', 'in_degree', 'out_degree', 'dep_impact', 'degree_centrality']
        categorical_features = []
        
        # Adicionar features categóricas se existirem
        for col in ['team', 'component', 'type']:
            if col in df.columns:
                df[col] = df[col].fillna('unknown')
                le = LabelEncoder()
                df[f'{col}_encoded'] = le.fit_transform(df[col])
                base_features.append(f'{col}_encoded')
                categorical_features.append(f'{col}_encoded')
                self.label_encoders[col] = le
        
        X = df[base_features]
        y = df['roi_ajustado']
        
        # Para datasets pequenos, usar abordagem mais simples
        if len(df) < 50:
            # Modelo simples sem CV extensivo
            self.model = lgb.LGBMRegressor(
                objective='regression',
                n_estimators=100,
                random_state=self.random_state,
                verbose=-1
            )
            
            self.model.fit(X, y)
            mae_cv = mean_absolute_error(y, self.model.predict(X))
            optimal_n_estimators = 100
            
        else:
            # Cross-validation para datasets maiores
            try:
                # Criar bins para estratificação baseada no target
                if len(y.unique()) < self.cv_folds:
                    from sklearn.model_selection import KFold
                    cv_splitter = KFold(n_splits=min(3, len(y)), shuffle=True, random_state=self.random_state)
                    split_iterator = cv_splitter.split(X)
                else:
                    y_bins = pd.qcut(y, q=min(self.cv_folds, len(y.unique())), labels=False, duplicates='drop')
                    cv_splitter = StratifiedKFold(n_splits=self.cv_folds, shuffle=True, random_state=self.random_state)
                    split_iterator = cv_splitter.split(X, y_bins)
                
                n_estimators_list = []
                mae_scores = []
                
                for train_idx, val_idx in split_iterator:
                    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                    
                    model = lgb.LGBMRegressor(
                        objective='regression',
                        random_state=self.random_state,
                        verbose=-1,
                        n_estimators=200
                    )
                    
                    # Treinar sem early stopping para evitar problemas
                    model.fit(X_train, y_train)
                    
                    n_estimators_list.append(model.n_estimators)
                    mae_scores.append(mean_absolute_error(y_val, model.predict(X_val)))
                
                # Modelo final
                optimal_n_estimators = max(100, int(np.mean(n_estimators_list)) if n_estimators_list else 200)
                mae_cv = np.mean(mae_scores) if mae_scores else 0.0
                
            except Exception as e:
                print(f"Erro no CV, usando modelo simples: {e}")
                optimal_n_estimators = 200
                mae_cv = 0.0
            
            # Treinar modelo final
            self.model = lgb.LGBMRegressor(
                objective='regression',
                n_estimators=optimal_n_estimators,
                random_state=self.random_state,
                verbose=-1
            )
            
            self.model.fit(X, y)
        
        # Feature importance
        try:
            self.feature_importance = dict(zip(base_features, self.model.feature_importances_))
        except:
            self.feature_importance = {feat: 1.0/len(base_features) for feat in base_features}
        
        # Predições
        df['score_ml'] = self.model.predict(X)
        
        return {
            'mae_cv': mae_cv,
            'n_estimators': optimal_n_estimators,
            'features': base_features
        }
    
    def topological_greedy_sort(self, df: pd.DataFrame, G: nx.DiGraph) -> pd.DataFrame:
        """Ordenação topológica gulosa baseada em score_ml"""
        result_df = df.copy()
        G_work = G.copy()
        
        ranked_items = []
        
        while G_work.nodes():
            # Encontrar nós sem dependências (in_degree == 0)
            available = [node for node in G_work.nodes() if G_work.in_degree(node) == 0]
            
            if not available:
                # Se há ciclos remanescentes, pegar qualquer nó
                available = list(G_work.nodes())
                print("Aviso: Ciclo detectado durante ordenação. Forçando progressão.")
            
            # Criar mapping de score_ml para os disponíveis
            available_scores = {}
            for node in available:
                node_data = result_df[result_df['id'].astype(str) == node].iloc[0]
                available_scores[node] = (
                    node_data['score_ml'],
                    node_data['dep_impact'],
                    -node_data['story_points'],  # Negativo para order crescente
                    node
                )
            
            # Escolher o melhor item (maior score_ml, depois critérios de desempate)
            best_item = max(available_scores.items(), key=lambda x: x[1])[0]
            
            ranked_items.append(best_item)
            
            # Remover item escolhido do grafo
            G_work.remove_node(best_item)
        
        # Atribuir ranks
        rank_mapping = {item: rank + 1 for rank, item in enumerate(ranked_items)}
        result_df['rank'] = result_df['id'].astype(str).map(rank_mapping)
        
        # Ordenar por rank
        result_df = result_df.sort_values('rank').reset_index(drop=True)
        
        return result_df
    
    def calculate_metrics(self, df: pd.DataFrame, G: nx.DiGraph) -> Dict:
        """Calcula métricas de qualidade da priorização"""
        metrics = {}
        
        # Top-20 dependency inversions
        top_20 = df.head(20)
        inversions = 0
        
        for _, row in top_20.iterrows():
            item_id = str(row['id'])
            item_rank = row['rank']
            
            # Verificar se alguma dependência tem rank maior
            for pred in G.predecessors(item_id):
                pred_rank = df[df['id'].astype(str) == pred]['rank'].iloc[0]
                if pred_rank > item_rank:
                    inversions += 1
        
        metrics['inversions_top_20'] = inversions
        
        # % do top-10 sem bloqueio
        top_10 = df.head(10)
        unblocked = top_10['deps_ok'].sum()
        metrics['pct_top10_unblocked'] = (unblocked / 10) * 100
        
        return metrics
    
    def prioritize_backlog(self, file_path: str) -> Tuple[pd.DataFrame, Dict]:
        """Executa todo o pipeline de priorização"""
        print("1. Lendo e padronizando dados...")
        df = self.read_and_standardize(file_path)
        print(f"   Carregados {len(df)} itens")
        
        print("2. Construindo grafo de dependências...")
        G, external_deps = self.build_dependency_graph(df)
        print(f"   Grafo: {len(G.nodes())} nós, {len(G.edges())} arestas")
        
        print("3. Calculando features do grafo...")
        df = self.calculate_graph_features(df, G)
        
        print("4. Treinando modelo de ML...")
        ml_metrics = self.train_ml_model(df)
        print(f"   MAE CV: {ml_metrics['mae_cv']:.4f}")
        
        print("5. Executando ordenação topológica gulosa...")
        df_ranked = self.topological_greedy_sort(df, G)
        
        print("6. Calculando métricas finais...")
        final_metrics = self.calculate_metrics(df_ranked, G)
        
        # Compilar métricas finais
        all_metrics = {
            'ml_mae_cv': ml_metrics['mae_cv'],
            'n_estimators': ml_metrics['n_estimators'],
            'features_used': ml_metrics['features'],
            'feature_importance': self.feature_importance,
            'inversions_top_20': final_metrics['inversions_top_20'],
            'pct_top10_unblocked': final_metrics['pct_top10_unblocked']
        }
        
        return df_ranked, all_metrics
    
    def save_results(self, df: pd.DataFrame, output_path: str) -> None:
        """Salva resultados em CSV"""
        # Selecionar e ordenar colunas de saída
        output_cols = [
            'rank', 'id', 'roi_ajustado', 'dep_impact', 'score_ml',
            'deps_ok', 'deps', 'in_degree', 'out_degree',
            'degree_centrality', 'story_points'
        ]
        
        # Adicionar colunas extras se existirem
        extra_cols = [col for col in df.columns if col not in output_cols and not col.endswith('_encoded')]
        output_cols.extend(extra_cols)
        
        # Filtrar apenas colunas existentes
        output_cols = [col for col in output_cols if col in df.columns]
        
        df[output_cols].to_csv(output_path, index=False)
        print(f"Resultados salvos em: {output_path}")

# Exemplo de uso
def main():
    """Exemplo de execução do algoritmo"""
    import os
    

    
    print("=== EXEMPLO DE PRIORIZAÇÃO DE BACKLOG ===\n")
    
    # Inicializar priorizador
    prioritizer = BacklogPrioritizer(
        decay_factor=0.6,
        cv_folds=3,  # Reduzido para dataset pequeno
        random_state=42
    )
    
    try:
        # Executar priorização
        df_result, metrics = prioritizer.prioritize_backlog('/workspaces/codespaces-jupyter/data/trilha_a.csv')
        
        # Salvar resultados
        prioritizer.save_results(df_result, '/workspaces/codespaces-jupyter/data/backlog_priorizado.csv')
        
        # Exibir resultados
        print("\n=== TOP 5 ITENS PRIORIZADOS ===")
        print(df_result[['rank', 'id', 'roi_ajustado', 'score_ml', 'deps_ok', 'story_points']].head())
        
        print("\n=== MÉTRICAS FINAIS ===")
        print(f"MAE Cross-Validation: {metrics['ml_mae_cv']:.4f}")
        print(f"Inversões Top-20: {metrics['inversions_top_20']}")
        print(f"% Top-10 sem bloqueio: {metrics['pct_top10_unblocked']:.1f}%")
        print(f"N° de estimadores usados: {metrics['n_estimators']}")
        
        print("\n=== IMPORTÂNCIA DAS FEATURES ===")
        for feature, importance in sorted(metrics['feature_importance'].items(), 
                                        key=lambda x: x[1], reverse=True):
            print(f"{feature}: {importance:.4f}")
        
        # Limpar arquivos de exemplo
        if os.path.exists('exemplo_backlog.csv'):
            os.remove('exemplo_backlog.csv')
            
        print(f"\n✅ Execução completa! Resultados salvos em 'exemplo_backlog_priorizado.csv'")
        
    except Exception as e:
        print(f"❌ Erro durante execução: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


=== EXEMPLO DE PRIORIZAÇÃO DE BACKLOG ===

1. Lendo e padronizando dados...
   Carregados 28 itens
2. Construindo grafo de dependências...
Ciclo detectado. Removida aresta: ('21', '22')
   Grafo: 28 nós, 31 arestas
3. Calculando features do grafo...
4. Treinando modelo de ML...
   MAE CV: 0.8571
5. Executando ordenação topológica gulosa...
6. Calculando métricas finais...
Resultados salvos em: /workspaces/codespaces-jupyter/data/backlog_priorizado.csv

=== TOP 5 ITENS PRIORIZADOS ===
   rank  id  roi_ajustado  score_ml  deps_ok  story_points
0     1  22             3      2.25     True             3
1     2  23             3      2.25    False             5
2     3   1             1      2.25     True             1
3     4   3             1      2.25    False             1
4     5  13             3      2.25     True             3

=== MÉTRICAS FINAIS ===
MAE Cross-Validation: 0.8571
Inversões Top-20: 0
% Top-10 sem bloqueio: 30.0%
N° de estimadores usados: 100

=== IMPORTÂNCIA DAS FEA